In [1]:
import json
from pathlib import Path
from typing import List


from dotenv import load_dotenv
from openai import OpenAI

load_dotenv()  # Load environment variables from .env file


True

In [2]:
import pandas as pd
import openai


In [3]:
# Training CSV File
train_csv1 = pd.read_csv('../data/train.csv')

In [4]:
test_csv = pd.read_csv('../data/test.csv')

In [5]:
train_csv1.head()


,id,text,label_text
0,1,I already made a transfer and want to cancel i...,cancel_transfer
1,2,I don't think I made this payment that is show...,card_payment_not_recognised
2,3,"You seem to forget there are so called ""rights...",not toxic
3,4,play the fifty songs i listen to most often,play_music
4,5,please make the smart socket turn off,iot


In [6]:
train_csv1.shape

(7994, 3)

In [7]:
pd.set_option('display.max_rows', 500)
pd.DataFrame(train_csv1.label_text.value_counts())

,count
label_text,
not toxic,1842
iot,374
play,291
play_music,288
weather,260
weather_query,260
news_query,212
news,212
datetime,191


In [8]:
# these are all the category labels in the training set
category_labels = train_csv1.label_text.unique()


def convert_to_gpt35_format_test(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"label_text": "' + row['label_text'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data

In [9]:
def convert_to_gpt35_format(dataset):
    fine_tuning_data = []
    for _, row in dataset.iterrows():
        json_response = '{"label_text": "' + row['label_text'] + '"}'
        fine_tuning_data.append({
            "messages": [
                {"role": "user", "content": row['text']},
                {"role": "assistant", "content": json_response}
            ]
        })
    return fine_tuning_data


#'train_data = train_csv
#'test_data = test_csv
converted_data= convert_to_gpt35_format(train_csv1)
#test_data = convert_to_gpt35_format(test_csv)
converted_data[0]['messages']
#test_data[0]['messages']
    

[{'role': 'user',
  'content': 'I already made a transfer and want to cancel it, how do I do that?'},
 {'role': 'assistant', 'content': '{"label_text": "cancel_transfer"}'}]

In [10]:
json.loads(converted_data[0]['messages'][-1]['content'])

{'label_text': 'cancel_transfer'}

In [11]:
from sklearn.model_selection import train_test_split

# Stratified splitting. Assuming 'Top Category' can be used for stratification
train_data, val_data = train_test_split(
    converted_data,
    test_size=0.01,
    random_state=42  # for reproducibility
)

In [12]:
train_data[0]


{'messages': [{'role': 'user',
   'content': 'how many hours difference is hong kong from new york city'},
  {'role': 'assistant', 'content': '{"label_text": "datetime"}'}]}

In [13]:
val_data[0]

{'messages': [{'role': 'user', 'content': 'start the radio now on low volume'},
  {'role': 'assistant', 'content': '{"label_text": "audio_volume_down"}'}]}

Create JSON file

In [14]:
def write_to_jsonl(data, file_path):
    with open(file_path, 'w') as file:
        for entry in data:
            json.dump(entry, file)
            file.write('\n')

training_file_name = "train.jsonl"
validation_file_name = "val.jsonl"

write_to_jsonl(train_data, training_file_name)
write_to_jsonl(val_data, validation_file_name)

In [15]:
type(train_data[0])

dict

In [16]:
type(val_data[0])

dict

In [17]:
from openai import OpenAI
client = OpenAI(api_key="Open-AI-API-Key")

In [18]:
# Upload Training and Validation Files
training_file = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)
validation_file = client.files.create(
    file=open(validation_file_name, "rb"), purpose="fine-tune"
)


In [19]:
print("training file id:", training_file.id)
print("validation file id:", validation_file.id)

training file id: file-CQprW4xUq7MCL3zYCN9KbLbO
validation file id: file-9VryKclwPUdWqQM9JuxXbbZu


create finetuning job

In [22]:
suffix_name = "xyz"

response = client.fine_tuning.jobs.create(
    training_file=training_file.id,
    validation_file=validation_file.id,
    model="gpt-3.5-turbo",
    suffix=suffix_name,
)
response

FineTuningJob(id='ftjob-BbZUnDMQKtjUqZm513z1dEXG', created_at=1716680439, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-fbKWUsxYuiRZV9Gwt9qNFmKm', result_files=[], seed=385631930, status='validating_files', trained_tokens=None, training_file='file-CQprW4xUq7MCL3zYCN9KbLbO', validation_file='file-9VryKclwPUdWqQM9JuxXbbZu', estimated_finish=None, integrations=[], user_provided_suffix='xyz')

In [23]:
client.fine_tuning.jobs.list(limit=10)

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-BbZUnDMQKtjUqZm513z1dEXG', created_at=1716680439, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=15, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-fbKWUsxYuiRZV9Gwt9qNFmKm', result_files=[], seed=385631930, status='validating_files', trained_tokens=None, training_file='file-CQprW4xUq7MCL3zYCN9KbLbO', validation_file='file-9VryKclwPUdWqQM9JuxXbbZu', estimated_finish=None, integrations=[], user_provided_suffix='xyz'), FineTuningJob(id='ftjob-xCj4f83NVYyYN9wuFK2YSlMj', created_at=1716680400, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=15, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-fbKWUsxYuiRZV9Gwt9qNFmKm', result_f

In [24]:
response = client.fine_tuning.jobs.retrieve("ftjob-V1mqbjkNT9Fg7gXflhO9vSjj")

In [25]:
response

FineTuningJob(id='ftjob-V1mqbjkNT9Fg7gXflhO9vSjj', created_at=1714780697, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:symbiosis:abc:9KxxNAOd', finished_at=1714784060, hyperparameters=Hyperparameters(n_epochs=3, batch_size=12, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-fbKWUsxYuiRZV9Gwt9qNFmKm', result_files=['file-4Mdkj8k9Z0NkpNBLoxIaZfnn'], seed=1970666179, status='succeeded', trained_tokens=779262, training_file='file-9EpAfO9WyroXOJ9MTj3ji23b', validation_file='file-IpCRhOCnFLexHo6TVs7PWFSL', estimated_finish=None, integrations=[], user_provided_suffix='abc')

In [26]:
fine_tuned_model_id = response.fine_tuned_model

In [27]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def format_test(row):
    formatted_message = [{"role": "user", "content": row['text']}]
    return formatted_message

def predict(test_messages, fine_tuned_model_id):
    response = client.chat.completions.create(
        model=fine_tuned_model_id, messages=test_messages, temperature=0, max_tokens=50
    )
    return response.choices[0].message.content

def store_predictions(test_df, fine_tuned_model_id):
    test_df['Prediction'] = None
    for index, row in test_df.iterrows():
        test_message = format_test(row)
        prediction_result = predict(test_message, fine_tuned_model_id)
        test_df.at[index, 'Prediction'] = prediction_result

    test_df.to_csv("predictions.csv")


In [28]:
#test_df = pd.read_csv("test.csv")
#store_predictions(test_df, fine_tuned_model_id)

In [29]:
# test_df.head(20)

In [30]:
test_csv = pd.read_csv('../data/test.csv')
test_csv.head(5)

,id,text
0,1,i don't want any alarms
1,2,remove new year from calendar
2,3,"I noticed an extra $1 charge on my statement, ..."
3,4,Help! I can't find my card.
4,5,Can I open an account for a child?


In [31]:
test_csv.shape

(4000, 2)

In [32]:
test_df= test_csv
test_df.head(5)

,id,text
0,1,i don't want any alarms
1,2,remove new year from calendar
2,3,"I noticed an extra $1 charge on my statement, ..."
3,4,Help! I can't find my card.
4,5,Can I open an account for a child?


In [33]:
store_predictions(test_df, fine_tuned_model_id)

In [34]:
test_df1.head(5)

NameError: name 'test_df1' is not defined

In [ ]:
test_df.shape

In [ ]:
test_df.head(5)


In [ ]:
test_df.head(20)

In [ ]:
# Save the results to a CSV file in required format
test_df.rename(columns={'Prediction':'label_text'},inplace=True)
submission = test_df[['id','label_text']]
submission.to_csv('submission_1.csv',index=False)

In [ ]:
submission

In [ ]:
x = pd.read_csv("submission_1.csv")

In [ ]:
x.head()

In [ ]:
x.info()


In [ ]:
# x.replace({'"label_text": ','' ,})



In [ ]:
x = x.label_text.replace({'{"label_text": "':''}, regex=True)

In [ ]:
type(x)


In [ ]:
x.head()
